# Body signal of smoking

## 1. Data Preprocessing

#### 1.1 Importing packages and modules

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import LabelEncoder, MinMaxScaler, RobustScaler, StandardScaler
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

import time

#### 1.2 데이터 불러오기

In [ ]:
smoking = pd.read_csv('../data/Smoking_raw/smoking.csv')
pd.set_option('display.max_columns',30)
smoking.head()

In [ ]:
smoking.info()

In [ ]:
print("\nThere are totally {} null values in the dataset".format(smoking.isnull().sum().sum())) # for checking null counts

#### 1.3 전처리 함수 생성

In [ ]:
def preprocessing(df_scaling):
    # Remove columns not necessary
    if 'ID' in df_scaling:
        df_scaling = df_scaling.drop("ID", axis = 1)
        if len(df_scaling.columns) == 1:
            return df_scaling
        
        # Oral(=구강검사 여부) 특성값은 모두 Y 값이므로 삭제.
        df_scaling = df_scaling.drop('oral', axis = 1) 
        
        # Categorical features label encoding
        cate_features = df_scaling[['gender','tartar']]

        lbe = LabelEncoder()
        lbe.fit_transform(df_scaling["gender"])
        df_scaling["gender"] = lbe.fit_transform(df_scaling["gender"])

        lbe = LabelEncoder()
        lbe.fit_transform(df_scaling["tartar"])
        df_scaling["tartar"] = lbe.fit_transform(df_scaling["tartar"])
        
        # hearing feature converting values 1, 2 => 1, 0
        df_scaling['hearing(left)'] = df_scaling['hearing(left)'].apply(lambda x: x-2 if x == 2.0 else x )
        df_scaling['hearing(right)'] = df_scaling['hearing(right)'].apply(lambda x: x-2 if x == 2.0 else x )
        
        # BMI 지수 계산 : bmi = kg/m^2
        df_scaling['bmi'] = df_scaling['weight(kg)']/((df_scaling['height(cm)']*0.01)**2)
        # wwi(비만 지수) 지수 계산 : wwi = cm/sqrt(kg)
        df_scaling['wwi'] = df_scaling['waist(cm)']/(df_scaling['weight(kg)'].apply(np.sqrt))

    return df_scaling

In [ ]:
df = preprocessing(smoking)
df

In [ ]:
# Check correlation value with heatmap
df.corr().style.background_gradient(cmap='magma')

**Competition Format**

In [ ]:
# competition_format
x_train = pd.read_csv('../data/Smoking_raw/competition_format/x_train.csv')
x_test = pd.read_csv('../data/Smoking_raw/competition_format/x_test.csv')
y_train = pd.read_csv('../data/Smoking_raw/competition_format/y_train.csv')
y_test = pd.read_csv('../data/Smoking_raw/competition_format/y_test.csv')

In [ ]:
x_train = preprocessing(x_train)
x_test = preprocessing(x_test)
y_train = preprocessing(y_train) # remove 'ID', 'oral' columns
y_test = preprocessing(y_test) # remove 'ID', 'oral' columns

#### 1.4 Scaler 함수 생성

In [ ]:
def scaling(train_data, test_data, scaled_form = 'MinMaxScaler()'):
    # 형태 별 특성 구분
    train_data = preprocessing(train_data)
    test_data = preprocessing(test_data)
    tr_cate_features = train_data[['gender', 'tartar', 'hearing(right)', 'hearing(left)', 'dental caries']]
    tr_scaled_features = train_data.drop(tr_cate_features.columns, axis=1)
    
    ts_cate_features = test_data[['gender', 'tartar', 'hearing(right)', 'hearing(left)', 'dental caries']]
    ts_scaled_features = test_data.drop(ts_cate_features.columns, axis=1)
    
    if scaled_form == 'StandardScaler()':
        # Standard scaler
        scaler = StandardScaler()
        scaler.fit(tr_scaled_features) # 훈련 데이터에 fit() 적용
        
        # 훈련 데이터와 테스트 데이터에 transform()을 통해 변환
        tr_scaled = scaler.transform(tr_scaled_features)
        ts_scaled = scaler.transform(ts_scaled_features)
        
        train_std_scaled = pd.DataFrame(tr_scaled, columns=tr_scaled_features.columns)
        train_std_scaled[tr_cate_features.columns] = tr_cate_features
        
        test_std_scaled = pd.DataFrame(ts_scaled, columns=ts_scaled_features.columns)
        test_std_scaled[ts_cate_features.columns] = ts_cate_features
        
        return train_std_scaled, test_std_scaled
    
    elif scaled_form == 'RobustScaler()':
        # Robust scaler
        scaler = RobustScaler()
        scaler.fit(tr_scaled_features) # 훈련 데이터에 fit() 적용
        
        # 훈련 데이터와 테스트 데이터에 transform()을 통해 변환
        tr_scaled = scaler.transform(tr_scaled_features)
        ts_scaled = scaler.transform(ts_scaled_features)
        
        # 데이터 프레임 형태로 변환
        train_robust_scaled = pd.DataFrame(tr_scaled, columns=tr_scaled_features.columns)
        train_robust_scaled[tr_cate_features.columns] = tr_cate_features
        
        test_robust_scaled = pd.DataFrame(ts_scaled, columns=ts_scaled_features.columns)
        test_robust_scaled[ts_cate_features.columns] = ts_cate_features
        
        return train_robust_scaled, test_robust_scaled
        
    else:
        # MinMax scaler
        scaler = MinMaxScaler()
        scaler.fit(tr_scaled_features) # 훈련 데이터에 fit() 적용
        
        # 훈련 데이터와 테스트 데이터에 transform()을 통해 변환
        tr_scaled = scaler.transform(tr_scaled_features)
        ts_scaled = scaler.transform(ts_scaled_features)
        
        # 데이터 프레임 형태로 변환
        train_mmx_scaled = pd.DataFrame(tr_scaled, columns = tr_scaled_features.columns)
        train_mmx_scaled[tr_cate_features.columns] = tr_cate_features
        
        test_mmx_scaled = pd.DataFrame(ts_scaled,columns = ts_scaled_features.columns)
        test_mmx_scaled[ts_cate_features.columns] = ts_cate_features

        return train_mmx_scaled, test_mmx_scaled

In [ ]:
x_train, x_test = scaling(x_train, x_test, 'StandardScaler()')
# x_train, x_test = scaling(x_train, x_test, 'RobustScaler()')
# x_train, x_test = scaling(x_train, x_test)

In [ ]:
x_train

In [ ]:
x_test

In [ ]:
# x_train 시각화
x_train.hist(figsize = (20, 20), bins = 12, legend=False)

In [ ]:
# x_test 시각화
x_test.hist(figsize = (20, 20), bins = 12, legend=False)

In [ ]:
sns.displot(
    data = df, x = "age", hue = "smoking",
    kind = "hist", height = 5, aspect = 1.5,
    palette="ch:rot=-.25,hue=1,light=.50").set(title =  "density relationship between 'age' and 'smoking' variables");


sns.displot(
    data = df, x = "systolic", hue = "smoking",
    kind = "kde", height = 5, aspect = 1.5,
    palette="ch:rot=-.25,hue=1,light=.50").set(title = "density relationship between 'systolic' and 'smoking' variables");


sns.displot(
    data = df, x = "waist(cm)", hue = "smoking",
    kind = "kde", height = 5, aspect = 1.5,
    palette="ch:rot=-.25,hue=1,light=.50").set(title = "density relationship between 'waist(cm)' and 'smoking' variables");

sns.displot(
    data = df, x = "bmi", hue = "smoking",
    kind = "kde", height = 5, aspect = 1.5,
    palette="ch:rot=-.25,hue=1,light=.50").set(title = "density relationship between 'bmi' and 'smoking' variables");

sns.displot(
    data = df, x = "wwi", hue = "smoking",
    kind = "kde", height = 5, aspect = 1.5,
    palette="ch:rot=-.25,hue=1,light=.50").set(title = "density relationship between 'wwi' and 'smoking' variables");

sns.displot(
    data = df, x = "triglyceride", hue = "smoking",
    kind = "kde", height = 5, aspect = 1.5,
    palette="ch:rot=-.25,hue=1,light=.50").set(title = "density relationship between 'wwi' and 'smoking' variables");

**Baseline : Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier # for modeling
from sklearn.metrics import accuracy_score, precision_score , recall_score

# 랜덤포레스트 객체 생성
rf_clf = RandomForestClassifier(random_state=0, n_estimators=2000)
rf_clf.fit(x_train, y_train)
rf_pred = rf_clf.predict(x_test)
rf_acc = accuracy_score(y_test, rf_pred)
rf_pred_probs = rf_clf.predict_proba(x_test)[:, 1]
print('랜덤 포레스트 정확도: {0:.4f}'.format(rf_acc))

In [ ]:
def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test,pred)
    accuracy = accuracy_score(y_test,pred)
    precision = precision_score(y_test,pred)
    recall = recall_score(y_test,pred)
    f1 = f1_score(y_test,pred)
    # ROC-AUC 추가 
    roc_auc = roc_auc_score(y_test,pred_proba)
    print('오차 행렬')
    print(confusion)
    # ROC-AUC print 추가
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

In [ ]:
get_clf_eval(y_test, rf_pred, rf_pred_probs)

## 2. XGBoost

#### General parameter
- booster
    - gbtree(tree based model) 또는 gblinear(linear model) 중 선택
    - Default = 'gbtree'
- silent
    - 출력 메시지 설정 관련 인수(나타내고 싶지 않을 경우 1로 설정)
    - Default = 1
- nthread
    - CPU 실행 스레드 개수 조정
    - Default는 전체 다 사용하는 것
    - 멀티코어/스레드 CPU 시스템에서 일부CPU만 사용할 때 변경

#### Boost Parameter
- eta: Learning rate (일반적으로 0.01 - 0.2)
    - 범위는 0 ~ 1로 클 수록 모형의 업데이트 속도는 빨라지지만, 과적합의 이슈 발생 가능성이 높음
- min_child_weight: min_child_weight를 기준으로 추가 분기 결정 (크면 Underfitting)
    - Default : 1
    - leaf node에 포함되는 최소 관측치의 수를 의미
    - 작은 값을 가질수록 과적합 발생 가능성이 높음 (과적합 조절 용도로 사용됨)
    - 범위: 0 ~ ∞
- max_depth: Tree 깊이 수
    - Default : 6
    - 트리의 최대 깊이를 설정
    - 0 을 지정하면 깊이의 제한이 없음
    - 과적합에 가장 민감하게 작용하는 파라미터 중 하나임 (과적합 조절 용도로 사용됨)
    - 범위: 0 ~ ∞
- max_leaf_node: 하나의 트리에서 node 개수
- gamma: split 하기 위한 최소의 loss 감소 정의
    - Default : 0
    - leaf node의 추가 분할을 결정할 최소손실 감소값
    - 해당 값보다 손실이 크게 감소할 때 분리
    - 값이 클수록 과적합 감소효과
    - 범위: 0 ~ ∞
- subsample
    - Default : 1
    - 학습 시 데이터 샘플링 비율을 지정(과적합 제어)
    - 일반적으로 0.5 ~ 1 사이의 값을 사용
    - 범위: 0 ~ 1
- colsample_bytree
    - Default : 1
    - 트리 생성에 필요한 feature의 샘플링에 사용
    - feature가 많을 때 과적합 조절에 사용
    - 범위: 0 ~ 1
- colsample_bylevel: 각 level마다 샘플링 비율
- lambda
    - Default : 1
    - L2 Regularization 적용 값
    - feature 개수가 많을 때 적용 검토
    - 클수록 과적합 감소 효과
- alpha: L1 norm
    - Default : 0
    - L1 Regularization 적용 값
    - feature 개수가 많을 때 적용 검토
    - 클수록 과적합 감소 효과

- scale_pos_weight: positive, negative weight 지정
    - Default : 1
    - 불균형 데이터셋의 균형을 유지

#### Train parameter
- objective	
    - reg:linear : 회귀
    - binary:logistic : 이진분류
    - multi:softmax : 다중분류, 클래스 반환
    - multi:softprob : 다중분류, 확률반환
- eval_metric : 검증에 사용되는 함수정의, 회귀 분석인 경우 'rmse'를, 클래스 분류 문제인 경우 'error'
    - rmse : Root Mean Squared Error
    - mae : mean absolute error
    - logloss : Negative log-likelihood
    - error : binary classification error rate
    - merror : multiclass classification error rate
    - mlogloss: Multiclass logloss
    - auc: Area Under Curve
- seed

### 2.1 Python wrapper를 사용한 XGB 모델

#### 2.1.1 DMarix 형태로 변환 후 분류기 생성

In [ ]:
# !pip install xgboost

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn.metrics import confusion_matrix, auc
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score, roc_curve

# adjusting data into DMatirx type
dtrain = xgb.DMatrix(data=x_train,label=y_train,feature_names=x_train.columns)
dtest = xgb.DMatrix(data=x_test,label=y_test,feature_names=x_train.columns)

In [ ]:
params = {'eta':0.1, # XGBClassifier일 경우 learning_rate 로 쓴다
          'objective':'binary:logistic', # 목적함수 : target 데이터가 0 or 1 이므로 이진 로지스틱 사용
          'eval_metric':'logloss', # 오류 함수의 평가 성능 지표 : logloss
          'early_stoppings':100, # 100회이상 시행시에도 오류가 내려가지않으면 중단
          'silent' : 0, # 출력 메시지 설정 관련 인수(나타내고 싶지 않을 경우 1로 설정)
          'verbosity':0}
          
num_rounds = 2000

In [ ]:
wlist = [(dtrain,'train'),(dtest,'eval')]
start = time.time()
xgb_model = xgb.train(params = params, dtrain=dtrain,num_boost_round= num_rounds,
                      early_stopping_rounds=100,evals = wlist)
end = time.time()
print("XGB 수행 시간: {0:.1f} 초 ".format(end - start)) # 29.5 sec
pred_probs = xgb_model.predict(dtest)
preds = [1 if x > 0.5 else 0 for x in pred_probs]
print('분류 결과 : {0:.1f} '.format(accuracy_score(y_test,preds)))

#### 2.1.2 평가지표 시각화(오차행렬, ROC curve)

In [ ]:
def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test,pred)
    accuracy = accuracy_score(y_test,pred)
    precision = precision_score(y_test,pred)
    recall = recall_score(y_test,pred)
    f1 = f1_score(y_test,pred)
    # ROC-AUC 추가 
    roc_auc = roc_auc_score(y_test,pred_proba)
    print('오차 행렬')
    print(confusion)
    # ROC-AUC print 추가
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

In [ ]:
get_clf_eval(y_test, preds, pred_probs)

In [ ]:
# Compute micro-average ROC curve and ROC area
fpr, tpr, _ = roc_curve(y_test.values, pred_probs)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(10,10))
lw = 2
plt.plot(fpr, tpr, color='red',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='blue', lw=lw, linestyle='--')
plt.xlim([-0.02, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,12)) # 축 반환
plot_importance(xgb_model,ax=ax)

In [ ]:
# !pip install graphviz

In [ ]:
# num_trees : 그림을 여러개 그릴시 그림 번호
# rankdir : 트리의 방향, 디폴트는 위아래 방향
# rankdir="LR" : 왼쪽에서 오른쪽 방향으로 트리를 보여준다.
xgb.plot_tree(xgb_model, num_trees=0, rankdir='LR')

fig = plt.gcf()
fig.set_size_inches(150, 100)

# 이미지 저장하고 싶다면
# fig.savefig('xgb_tree_py_wp.png')

plt.show()

### 2.2 Scikit-leran wrapper를 사용한 XGB 모델

#### 2.2.1 분류기 생성

In [ ]:
# # Initiate XGBoost Classifier
# xgb_clf = XGBClassifier(learning_rate=0.1,n_estimators=2000,max_depth=3,
#                         silent=True,
#                         objective='binary:logistic',
#                         booster='gbtree',
#                         n_jobs=1,
#                         nthread=None,
#                         gamma=0,
#                         min_child_weight=1,
#                         max_delta_step=0,
#                         subsample=1,
#                         colsample_bytree=1,
#                         colsample_bylevel=1,
#                         reg_alpha=0,
#                         reg_lambda=1,
#                         scale_pos_weight=1,
#                         base_score=0.5,
#                         random_state=0,
#                         seed=None,
#                         missing=None)
# # Print default setting
# xgb_clf.get_params()

In [ ]:
# Train the model
start = time.time()
xgb_clf = XGBClassifier(learning_rate=0.1,n_estimators=2000,max_depth=3,
                        silent=0,
                        objective='binary:logistic',
                        booster='gbtree',
                        n_jobs=1,
                        nthread=None,
                        gamma=0).fit(x_train,y_train)
# Make prediction
xgb_pred = xgb_clf.predict(x_test)
# Get predicted probability
xgb_pred_probs = xgb_clf.predict_proba(x_test)[:,1]
end = time.time()
print("XGB 수행 시간: {0:.1f} 초 ".format(end - start)) # 
print('분류 결과 : {0:.1f} '.format(accuracy_score(y_test,xgb_pred)))

#### 2.2.2 평가지표 시각화(오차행렬, ROC curve)

In [ ]:
get_clf_eval(y_test, xgb_pred, xgb_pred_probs)

In [ ]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(xgb_clf, x_test, y_test, cmap = plt.cm.Blues, normalize = "true")

In [ ]:
# Compute micro-average ROC curve and ROC area
fpr, tpr, _ = roc_curve(y_test.values, xgb_pred_probs)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(10,10))
lw = 2
plt.plot(fpr, tpr, color='red',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='blue', lw=lw, linestyle='--')
plt.xlim([-0.02, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,12)) # 축 반환
plot_importance(xgb_clf,ax=ax)

In [ ]:
# num_trees : 그림을 여러개 그릴시 그림 번호
# rankdir : 트리의 방향, 디폴트는 위아래 방향
# rankdir="LR" : 왼쪽에서 오른쪽 방향으로 트리를 보여준다.
xgb.plot_tree(xgb_clf, num_trees=0, rankdir='LR')

fig = plt.gcf()
fig.set_size_inches(150, 100)

# 이미지 저장하고 싶다면
# fig.savefig('xgb_tree_skl_wp.png')

plt.show()

#### 2.2.3 Grid Search를 이용하여 최적 하이퍼 파라미터 탐색

In [ ]:
from sklearn.model_selection import GridSearchCV
# xgb 모델 생성
xgb = XGBClassifier(learning_rate=0.1,n_estimators=2000,
                    gamma=0, 
                    subsample=0.8, # 각 트리마다의 관측 데이터 샘플링 비율, default = 1, 일반적으로 0.5 ~ 1
                    colsample_bytree=0.8, # 각 트리마다의 feature 샘플링 비율, default = 1, 일반적으로 0.5 ~ 1
                    objective= 'binary:logistic', 
                    verbose=10)

# parameter 들을 dictionary 형태로 설정
xgb_params = { 
              'max_depth':range(3,13,3), 
              'min_child_weight':range(1,6,2)
             }

In [ ]:
start = time.time()
# GridSearch를 통해 최적 hyperparameter를 검색
clf = GridSearchCV(xgb,xgb_params,scoring='accuracy',cv=5)
clf.fit(x_train, y_train)
print(clf.best_params_)
pred = clf.predict(x_test)
print('분류 결과 : {0:.1f} '.format(accuracy_score(y_test,pred)))
end = time.time()
print("XGB 수행 시간: {0:.1f} 초 ".format(end - start))

In [ ]:
# xgb 모델 생성
xgb = XGBClassifier(learning_rate=0.1, 
                    n_estimators=2000,
                    max_depth=3,
                    min_child_weight=3,
                    gamma=0, 
                    subsample=0.8, # 각 트리마다의 관측 데이터 샘플링 비율, default = 1, 일반적으로 0.5 ~ 1
                    colsample_bytree=0.8, # 각 트리마다의 feature 샘플링 비율, default = 1, 일반적으로 0.5 ~ 1
                    objective= 'binary:logistic', 
                    verbose=10)

# parameter 들을 dictionary 형태로 설정
xgb_params = { 
              'gamma':[i/10.0 for i in range(0,5)]
             }

In [ ]:
start = time.time()
# GridSearch를 통해 최적 hyperparameter를 검색
clf = GridSearchCV(xgb,xgb_params,scoring='accuracy',cv=5)
clf.fit(x_train, y_train)
print(clf.best_params_)
pred = clf.predict(x_test)
print('분류 결과 : {0:.1f} '.format(accuracy_score(y_test,pred)))
end = time.time()
print("XGB 수행 시간: {0:.1f} 초 ".format(end - start))

In [ ]:
# xgb 모델 생성
xgb = XGBClassifier(learning_rate=0.1, 
                    n_estimators=2000,
                    max_depth=3,
                    min_child_weight=3,
                    gamma=0, 
                    subsample=0.8, # 각 트리마다의 관측 데이터 샘플링 비율, default = 1, 일반적으로 0.5 ~ 1
                    colsample_bytree=0.8, # 각 트리마다의 feature 샘플링 비율, default = 1, 일반적으로 0.5 ~ 1
                    objective= 'binary:logistic', 
                    verbose=10)

# parameter 들을 dictionary 형태로 설정
xgb_params = { 
              'subsample':[i/10.0 for i in range(6,10)],
              'colsample_bytree':[i/10.0 for i in range(6,10)]
             }

In [ ]:
start = time.time()
# GridSearch를 통해 최적 hyperparameter를 검색
clf = GridSearchCV(xgb,xgb_params,scoring='accuracy',cv=5)
clf.fit(x_train, y_train)
print(clf.best_params_)
pred = clf.predict(x_test)
print('분류 결과 : {0:.1f} '.format(accuracy_score(y_test,pred)))
end = time.time()
print("XGB 수행 시간: {0:.1f} 초 ".format(end - start))

In [ ]:
# Train the model
start = time.time()
xgb_clf = XGBClassifier(learning_rate=0.1, 
                        n_estimators=2000,
                        max_depth=6,
                        min_child_weight=3,
                        gamma=0, 
                        subsample=0.9, # 각 트리마다의 관측 데이터 샘플링 비율, default = 1, 일반적으로 0.5 ~ 1
                        colsample_bytree=0.8, # 각 트리마다의 feature 샘플링 비율, default = 1, 일반적으로 0.5 ~ 1
                        objective= 'binary:logistic', 
                        verbose=10).fit(x_train,y_train)
# Make prediction
xgb_pred = xgb_clf.predict(x_test)
# Get predicted probability
xgb_pred_probs = xgb_clf.predict_proba(x_test)[:,1]
end = time.time()
print("XGB 수행 시간: {0:.1f} 초 ".format(end - start)) # 
print('분류 결과 : {0:.1f} '.format(accuracy_score(y_test,xgb_pred)))

#### 2.2.4 하이퍼파라미터 수정 후, 평가지표 시각화(오차행렬, ROC curve)

In [ ]:
get_clf_eval(y_test, xgb_pred, xgb_pred_probs)

In [ ]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(xgb_clf, x_test, y_test, cmap = 'summer', normalize = "true")

In [ ]:
import xgboost as xgb

# num_trees : 그림을 여러개 그릴시 그림 번호
# rankdir : 트리의 방향, 디폴트는 위아래 방향
# rankdir="LR" : 왼쪽에서 오른쪽 방향으로 트리를 보여준다.
xgb.plot_tree(xgb_clf, num_trees=0, rankdir='LR')

fig = plt.gcf()
fig.set_size_inches(150, 100)

# 이미지 저장하고 싶다면
# fig.savefig('xgb_clf_tree.png')

plt.show()

In [ ]:
# Compute micro-average ROC curve and ROC area
fpr, tpr, _ = roc_curve(y_test.values, xgb_pred_probs)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(10,10))
lw = 2
plt.plot(fpr, tpr, color='red',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='blue', lw=lw, linestyle='--')
plt.xlim([-0.02, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve')
plt.legend(loc="lower right")
plt.show()

## 3. Light GBM

#### Parameters
- num_iterations : 수행 반복 횟수
    - Default : 100)
    - 학습에 활용될 weak learner의 반복 수
- learning_rate : 학습률
    - Default : 0.1
    - 일반적으로 확습률(learning rate)로 불리우는 파라미터
    - weak learner의 반영 수준을 나타냄
    - 범위는 0 ~ 1로 클 수록 모형의 업데이트 속도는 빨라짐. 클 수록 과적합의 이슈 발생 가능성이 높음
- max_depth : 최대 깊이
    - Default : -1
    - 트리의 최대 깊이를 설정
    - 0을 지정하면 깊이의 제한이 없음
    - 과적합에 가장 민감하게 작용하는 파라미터 중 하나임 (과적합 조절 용도로 사용됨)
    - 범위: 0 ~ ∞
- min_data_in_leaf : 노드 별 leaf 최소 개수
    - Default : 20
    - 최종 leaf node가 되기 위한 최소 레코드 수
    - 과적합 제어용으로 활용
- num_leaves
    - Default : 31
    - 하나의 트리가 가지는 최대 leaf 수
- boosting
    - Default : 'gbdt'
    - 실행하고자 하는 알고리즘 타입 정의
        - gdbt : Gradient Boosting Decision Tree
        - rf : Random Forest
        - dart : Dropouts meet Multiple Additive Regression Trees
        - goss : Gradient-based One-Side Sampling
- bagging_fraction
    - Default : 1.0)
    - 데이터 샘플링 비율
    - 과적합 제어용
- feature_fraction
    - Default 1.0)
    - 개별 트리 학습 시 선택하는 feature 비율
    - 과적합 제어용
- lamda_l2
    - Default : 0
    - L2 Regularization 적용 값
    - feature 개수가 많을 때 적용 검토
    - 클수록 과적합 감소 효과
- lamda_l1
    - Default : 0
    - L1 Regularization 적용 값
    - feature 개수가 많을 때 적용 검토
    - 클수록 과적합 감소 효과
- objective
    - Default : regression
        - regression : 회귀
        - binary : 이진분류
        - multiclass : 다중분류
- early_stopping_round
    - Default : 0
    - 이전 학습 대비 일정 수준 이상의 성능 효과가 없을 시 학습을 중단함
    - 지나친 iteration을 줄이는데 도움이 되기 때문에, 학습 속도를 높일 수 있음
- min_gain_to_split
    - Default : 0
    - 트리를 분기하기 위해 필요한 최소한의 gain
- max_cat_threshold
    - Default : 32
    - 카테고리 그룹을 정의된 수로 합치고 그룹 경계선에서 분기 포인트 searching
    - 카테고리 수가 클 때, 과적합을 방지하는 분기 포인트 searching

### 3.1 분류기 생성

In [ ]:
# !pip install lightgbm

In [ ]:
import lightgbm as lgb
from lightgbm import LGBMClassifier
from lightgbm import plot_importance # for visualization about features' importance by using plot_importance()

lgbm_clf = LGBMClassifier(n_estimators=2000)
evals = [(x_test,y_test)]

start = time.time()

# 학습 : 조기중단 수행(100)
lgbm_clf.fit(x_train,y_train, 
             early_stopping_rounds=100, 
             eval_metric='logloss', 
             eval_set=evals, 
             verbose=True)
                

# 예측
preds = lgbm_clf.predict(x_test)
pred_probs = lgbm_clf.predict_proba(x_test)[:, 1]
end = time.time()
print("LGBM 수행 시간: {0:.1f} 초 ".format(end - start)) # 10.1 sec
print('분류 결과 : {0:.1f} '.format(accuracy_score(y_test,preds)))

In [ ]:
# lgbm_clf.get_params()

### 3.2 평가지표 시각화(오차행렬, ROC curve)

In [ ]:
def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test,pred)
    accuracy = accuracy_score(y_test,pred)
    precision = precision_score(y_test,pred)
    recall = recall_score(y_test,pred)
    f1 = f1_score(y_test,pred)
    # ROC-AUC 추가 
    roc_auc = roc_auc_score(y_test, pred_proba)
    print('오차 행렬')
    print(confusion)
    # ROC-AUC print 추가
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

In [ ]:
get_clf_eval(y_test, preds, pred_probs)

In [ ]:
plot_confusion_matrix(lgbm_clf, x_test, y_test, cmap = plt.cm.Greens, normalize = "true")

In [ ]:
# Compute micro-average ROC curve and ROC area
fpr, tpr, _ = roc_curve(y_test.values, pred_probs)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(10,10))
lw = 2
plt.plot(fpr, tpr, color='red',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='blue', lw=lw, linestyle='--')
plt.xlim([-0.02, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 12))
plot_importance(lgbm_clf, ax=ax,importance_type='split')

In [ ]:
import lightgbm as lgb
lgb.plot_tree(lgbm_clf)

fig = plt.gcf()
fig.set_size_inches(150, 100)

# 이미지 저장하고 싶다면
# fig.savefig('lgbm_tree.png')

plt.show()

### 3.3 Grid Search를 이용하여 최적 하이퍼파라미터 탐색

In [ ]:
from sklearn.model_selection import GridSearchCV

# Generate the Light GBM model
lgbm_clf = LGBMClassifier(learning_rate=0.1,n_estimators=2000,
                           subsample=0.8, # 각 트리마다의 관측 데이터 샘플링 비율, default = 1, 일반적으로 0.5 ~ 1
                           colsample_bytree=0.8, # 각 트리마다의 feature 샘플링 비율, default = 1, 일반적으로 0.5 ~ 1
                           verbose=10)

# parameter 들을 dictionary 형태로 설정
lgbm_params = {'num_leaves':[33,65,97,129], #  num_leaves = 2^(max_depth)는 depth-wise tree와 같은 수의 leaves를 가지게 하여, 이보다 작게 설정해야 오버피팅을 줄일 수 있다.
               'max_depth':[-1,3,6,9,12], 
               'min_child_samples':[5,10,15],
               'min_child_weight':[1,3,5,7],
               'reg_alpha':[0,0.01,0.03]}

In [ ]:
start = time.time()
# GridSearch를 통해 최적 hyperparameter를 검색
grid_lgbm_clf = GridSearchCV(lgbm_clf,lgbm_params,scoring='accuracy',cv=5)
grid_lgbm_clf.fit(x_train, y_train)
print(grid_lgbm_clf.best_params_)
pred = grid_lgbm_clf.predict(x_test)
print('분류 결과 : {0:.1f} '.format(accuracy_score(y_test,pred)))
end = time.time()
print("LGBM 수행 시간: {0:.1f} 초 ".format(end - start))

In [ ]:
lgbm_clf_1 = LGBMClassifier(n_estimators=2000,max_depth=3,min_child_smaples=1,min_child_weight=5,num_leaves=20,reg_alpha=0)
evals = [(x_test,y_test)]

start = time.time()
# 학습 : 조기중단 수행(100)
lgbm_clf_1.fit(x_train,y_train,early_stopping_rounds=100,
                eval_metric='logloss',eval_set=evals, verbose=True)

# 예측
preds = lgbm_clf_1.predict(x_test)
pred_proba = lgbm_clf_1.predict_proba(x_test)[:, 1]
end = time.time()
print("LGBM 수행 시간: {0:.1f} 초 ".format(end - start))
print('분류 결과 : {0:.1f} '.format(accuracy_score(y_test,preds)))

## 4. Scaler 별 결과 확인

## 5. 이상치 탐색(with IQR) 및 제거 후 결과 확인

**Outlier detection with IQR**

In [ ]:
from collections import Counter

def outlier_detection(df, n, columns):
    rows = []
    will_drop_train = []
    for col in columns:
        Q1 = np.nanpercentile(df[col], 25)
        Q3 = np.nanpercentile(df[col], 75)
        IQR = Q3 - Q1
        outlier_point = 1.5 * IQR
        rows.extend(df[(df[col] < Q1 - outlier_point)|(df[col] > Q3 + outlier_point)].index)
    for r, c in Counter(rows).items():
        if c >= n: will_drop_train.append(r)
    return will_drop_train

will_drop_train = outlier_detection(df, 5, df.select_dtypes(["float", "int"]).columns)
will_drop_train[0:5]

In [ ]:
will_drop_train = outlier_detection(x_train, 5, x_train.select_dtypes(["float", "int"]).columns)
will_drop_train[0:5]

In [ ]:
x_train.drop(will_drop_train, inplace = True, axis = 0)
y_train.drop(will_drop_train, inplace = True, axis = 0)

In [ ]:
df.drop(will_drop_train, inplace = True, axis = 0)
df

### 5.1 XGB : Python wrapper

In [ ]:
# after deleting outlier data

wlist = [(dtrain,'train'),(dtest,'eval')]
start = time.time()
xgb_model = xgb.train(params = params, dtrain=dtrain,num_boost_round= num_rounds,
                      early_stopping_rounds=100,evals = wlist)
end = time.time()
print("XGB 수행 시간: {0:.1f} 초 ".format(end - start)) # 29.1 sec
pred_probs = xgb_model.predict(dtest)
preds = [1 if x > 0.5 else 0 for x in pred_probs]
print('분류 결과 : {0:.1f} '.format(accuracy_score(y_test,preds)))

In [ ]:
def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test,pred)
    accuracy = accuracy_score(y_test,pred)
    precision = precision_score(y_test,pred)
    recall = recall_score(y_test,pred)
    f1 = f1_score(y_test,pred)
    # ROC-AUC 추가 
    roc_auc = roc_auc_score(y_test,pred_proba)
    print('오차 행렬')
    print(confusion)
    # ROC-AUC print 추가
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

In [ ]:
get_clf_eval(y_test, preds, pred_probs)

In [ ]:
# num_trees : 그림을 여러개 그릴시 그림 번호
# rankdir : 트리의 방향, 디폴트는 위아래 방향
# rankdir="LR" : 왼쪽에서 오른쪽 방향으로 트리를 보여준다.
xgb.plot_tree(xgb_model, num_trees=0, rankdir='LR')

fig = plt.gcf()
fig.set_size_inches(150, 100)

# 이미지 저장하고 싶다면
# fig.savefig('xgb_tree_py_wp.png')

plt.show()

### 5.2 XGB : Scikit-learn wrapper

In [ ]:
# after deleting the outlier data
# Train the model
start = time.time()
xgb_clf = XGBClassifier(learning_rate=0.1,n_estimators=2000,max_depth=3,
                        silent=True,
                        objective='binary:logistic',
                        booster='gbtree',
                        n_jobs=1,
                        nthread=None,
                        gamma=0).fit(x_train,y_train)
# Make prediction
xgb_pred = xgb_clf.predict(x_test)
# Get predicted probability
xgb_pred_probs = xgb_clf.predict_proba(x_test)[:,1]
end = time.time()
print("XGB 수행 시간: {0:.1f} 초 ".format(end - start)) # 
print('분류 결과 : {0:.1f} '.format(accuracy_score(y_test,xgb_pred)))

In [ ]:
get_clf_eval(y_test, xgb_pred, xgb_pred_probs)
# 평가 지표의 수치가 하락함